In [1]:
# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import time
import json

from sklearn.neural_network import MLPClassifier

In [3]:
from data_helpers.wine_quality_data_helper import load_wine_quality_data
from data_helpers.mnist_data_helper import load_mnist_data

2024-03-25 22:46:17.153286: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 22:46:17.791668: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
DATASET_NAME = 'mnist'
# DATASET_NAME = 'wine_quality'

SAVE = True
K_FOLDS = 5

In [5]:
if DATASET_NAME == 'wine_quality':
    DATASET_STR = 'Wine Quality'
    X_train, y_train, X_test, y_test = load_wine_quality_data()
elif DATASET_NAME == 'mnist':
    DATASET_STR = 'MNIST'
    X_train, y_train, X_test, y_test = load_mnist_data()
else:
    raise ValueError(f'Invalid dataset name {DATASET_NAME}')

In [6]:
# get the number of instances of each label in y
for i in np.unique(y_train):
    print(f'Label {i}: {np.sum(y_train == i)} instances')

Label 0: 5923 instances
Label 1: 6742 instances
Label 2: 5958 instances
Label 3: 6131 instances
Label 4: 5842 instances
Label 5: 5421 instances
Label 6: 5918 instances
Label 7: 6265 instances
Label 8: 5851 instances
Label 9: 5949 instances


In [7]:
df = pd.DataFrame()
n = X_train.shape[0] // K_FOLDS

for i in tqdm(range(K_FOLDS)):
    X_train_k = np.concatenate([X_train[:i * n], X_train[(i + 1) * n:]])
    y_train_k = np.concatenate([y_train[:i * n], y_train[(i + 1) * n:]])
    X_test_k = X_train[i * n:(i + 1) * n]
    y_test_k = y_train[i * n:(i + 1) * n]

    model = MLPClassifier(hidden_layer_sizes=[], max_iter=1000, random_state=i, early_stopping=True, n_iter_no_change=10, verbose=False)

    t0 = time.perf_counter()
    model.fit(X_train_k, y_train_k)
    t1 = time.perf_counter()
    training_time = t1 - t0

    t0 = time.perf_counter()
    y_test_pred = model.predict(X_test_k) 
    t1 = time.perf_counter()
    evaluation_time = t1 - t0

    df.loc[i, 'training_time'] = training_time
    df.loc[i, 'evaluation_time'] = evaluation_time
    df.loc[i, 'fold'] = i
    df.loc[i, 'training_curve'] = json.dumps(model.loss_curve_)
    df.loc[i, 'validation_scores'] = json.dumps(model.validation_scores_)

100%|██████████| 5/5 [01:28<00:00, 17.62s/it]


In [8]:
df_path = f'results/step_4/{DATASET_NAME}/baseline_metrics.csv'
os.makedirs(os.path.dirname(df_path), exist_ok=True)
df.to_csv(df_path, index=False)